In [1]:
# 1. 라이브러리 및 도구 불러오기
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.tools import tool
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

# 도구 1: 메뉴 DB 검색
@tool
def db_search_cafe_func(query: str) -> list:
    """로컬 벡터 DB에서 카페 메뉴 정보를 검색합니다."""
    db = FAISS.load_local(
        "./db/cafe_db", 
        OllamaEmbeddings(model="bge-m3"), 
        allow_dangerous_deserialization=True
    )
    return db.similarity_search(query, k=2)

# 도구 2: 위키 요약
@tool
def wiki_summary(topic: str) -> str:
    """위키피디아에서 요약 정보를 검색합니다."""
    wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
    return wiki.run(topic)

# 도구 3: 웹 검색
@tool
def tavily_search_func(query: str) -> str:
    """웹 검색을 수행합니다."""
    tavily = TavilySearchResults(k=3)
    return str(tavily.run(query))

In [3]:
from langchain.prompts import FewShotPromptTemplate, ChatPromptTemplate, PromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# 시스템 메시지
system_message = SystemMessage(content="""
당신은 카페 메뉴 정보와 일반적인 음식/음료 지식을 제공하는 AI입니다.

도구 사용 가이드라인:
- db_search_cafe_func: 카페 메뉴 정보 (가격, 재료, 설명)
- wiki_summary: 일반 지식 (역사, 제조법, 문화적 배경)
- tavily_search_func: 최신 정보 (트렌드, 뉴스, 실시간 정보)

사용 원칙:
1. 카페 메뉴 관련 질문 → 반드시 메뉴 DB 먼저 검색
2. 역사/문화/일반 지식 → 위키피디아 활용
3. 최신 트렌드/뉴스 → 웹 검색 활용
4. 복합 질문 → 여러 도구 순차 사용
5. 정보 출처를 명확히 구분하여 답변
""")

# 예시 템플릿
example_template = PromptTemplate.from_template("""Human: {input}
AI: 도구들을 활용해 정보를 수집하겠습니다.
AI: [db_search_cafe_func 도구 호출]
AI: [wiki_summary 도구 호출]
AI: {output}
""")

# 예시들
examples = [
    {
        "input": "아메리카노 정보와 커피 역사를 알려주세요",
        "output": "아메리카노는 에스프레소에 뜨거운 물을 더한 블랙커피로, ₩4,500입니다. 원두 본연의 맛이 특징입니다. 커피의 역사는 15세기 에티오피아에서 시작되었습니다."
    }
]

# Few-shot 프롬프트 생성
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_template,
    prefix="다음은 사용자 질문에 따라 도구를 선택하고 정보를 조합하는 예제입니다.",
    suffix="Human: {input}\nAI:",
    input_variables=["input"]
)

# 전체 메시지 구성
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    HumanMessage(content="{input}")
])

In [4]:
# 3. LLM과 도구 연결 (GPT-3.5 기준)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

tools = [tavily_search_func, wiki_summary, db_search_cafe_func]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type="openai-tools",
    verbose=True
)

C:\Users\hun\AppData\Local\Temp\ipykernel_8136\3418842977.py:6: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [5]:
# 4. 복합 질문 실행
query = "카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래도 알려줘."
response = agent.run(query)
print(response)

C:\Users\hun\AppData\Local\Temp\ipykernel_8136\4251739661.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(query)




> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}